# LangChain v0.3 - Einführung in die Nutzung mit Beispielen in Python.

In [ ]:
#pip install langchain
#pip install langchain-core

Um die Dienste von OpenAI und TogetherAI nutzen zu können, benötigen wir einen API-Key. Nach der Registrierung bei den jeweiligen Anbietern findet man diesen Schlüssel meist im Benutzer-Dashboard (oder einem ähnlichen Bereich). Der Key ist in der Regel eine längere Sequenz aus Zahlen und Buchstaben.

Um Missbrauch zu verhindern, sollten diese Keys nicht direkt im Code gespeichert werden. Stattdessen speichern wir sie als Umgebungsvariablen in einer `.env`-Datei.

Um die Umgebungsvariablen aus einer `.env`-Datei in die Systemumgebung zu laden, verwenden wir die Funktion `load_dotenv()`.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Chat Models

Um mit OpenAI-Modellen über LangChain arbeiten zu können, müssen wir das Paket `langchain-openai` installieren:

In [2]:
#pip install langchain-openai

Wir werden nur mit Chat-Modellen arbeiten. Dafür brauchen wir die Klasse `ChatOpenAI` importieren.

Weitere Informationen zur Klasse findest du [hier](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html).

In [2]:
from langchain_openai import ChatOpenAI

Im nächsten Schritt initialisieren wir das OpenAI-Modell mit der `ChatOpenAI`-Klasse. Hier wird das Modell `gpt-4o-mini` verwendet, das ein kleineres und schnelleres Modell ist. 

Die Temperatur ist auf 0 gesetzt, was bedeutet, dass das Modell deterministischer antwortet und weniger zufällige Variationen in den Ausgaben erzeugt.

(Der API-Schlüssel wird bereits über Umgebungsvariablen bereitgestellt und daher nicht erneut im Code spezifiziert werden muss.)

In [3]:
llm_OpenAI_4oMini = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

Unser erstes kurzes Beispiel ist eine Übersetzung von Englischen ins Deutsche. 

Um das Modell anzusprechen, nutzen wir die Methode `invoke()`.

In [ ]:
result = llm_OpenAI_4oMini.invoke("""You are a helpful assistant that translates English to German. Please translate the user sentence.
User sentence:
```
I love Digital Humanities.
```
""")
print(result)

Der Output zeigt die Antwort des Modells sowie relevante Metadaten als Objekt mit mehreren Attributen, die Dictionaries enthalten:

- `content`: Die eigentliche Antwort des Modells.
- `response_metadata`: Metadaten zur Anfrage, wie die Anzahl der verwendeten Tokens, der Modellname (z. B. `gpt-4o-mini-2024-07-18`), und der Grund, warum die Ausgabe gestoppt wurde (`finish_reason='stop'`).
- `usage_metadata`: Informationen zur Anzahl der Eingabe- und Ausgabetokens, die bei der Anfrage verwendet wurden.

Dieses Objekt liefert uns also nicht nur die Antwort des Modells, sondern auch nützliche Daten zur Token-Nutzung und Modellleistung.



In [ ]:
print(type(result))

Objekt der Klasse `<class 'langchain_core.messages.ai.AIMessage'>`

In [ ]:
print(result.content)

In der obigen Anfrage wurde der Prompt als einfacher String übergeben. Eine andere Möglichkeit ist, einen Prompt mit strukturierten Nachrichten und definierten Rollen zu verwenden.

Vorteil dieses Ansatzes: **Bessere Kontextualisierung**. Durch die Definition von Rollen kann das Modell besser verstehen, in welchem Kontext es agieren soll (auch wenn dies bei einfachen Beispielen oft keinen spürbaren Unterschied macht).

In [ ]:
messages = [
    (
        "system", "You are a helpful assistant that translates English to German. Please translate the user sentence."
    ),
    ("human", "I love Digital Humanities."),
]
result = llm_OpenAI_4oMini.invoke(messages)
print(result)

In [ ]:
result.content

Wie sieht das zum Beispiel mit einem anderen Modell aus?

LangChain unterstützt viele verschiedene Anbieter. Hier ist eine Liste der unterstützten Chat-Modelle: https://python.langchain.com/docs/integrations/chat/#advanced-features

Wir werden mit TogetherAI weiterarbeiten. 
Liste der Modelle von TogetherAI: https://api.together.xyz/models

Versuchen wir es mit Llama.


Aber zunächst müssen wir das Paket `langchain-together` installieren:

In [10]:
#pip install langchain-together

In [9]:
from langchain_together import ChatTogether

In [10]:
llm_TogehterAI_Llama3_8B = ChatTogether(
    model="meta-llama/Meta-Llama-3-8B-Instruct-Turbo",
    temperature=0,
)

Hier ein weiteres Beispiel für die Übersetzung ins Deutsche, diesmal mit TogetherAI:

In [ ]:
result = llm_TogehterAI_Llama3_8B.invoke("""You are a helpful assistant that translates English to German. Please translate the user sentence.
User sentence:
```
I love Digital Humanities.
```
""")
print(result.content)

Die Antwort ist ähnlich, jedoch mit zusätzlichen Inhalten – einem einleitenden Satz und einer zusätzlichen Anmerkung (Note).

Und jetzt mit Messages

In [ ]:
messages = [
    (
        "system", "You are a helpful assistant that translates English to German. Please translate the user sentence."
    ),
    ("human", "I love Digital Humanities."),
]
result = llm_TogehterAI_Llama3_8B.invoke(messages)
print(result.content)

Die Antwort in dieser Variante ist kompakter und entspricht eher dem, was wir uns meistens wünschen – einer Antwort ohne Zusatzinformationen.

*Randbemerkung: Für diese Präsentation ist es nicht wichtig, ob die richtige Übersetzung "Digital Humanities" oder "digitale Geisteswissenschaften" lautet. Es soll lediglich das unterschiedliche Verhalten gezeigt werden, das durch verschiedene Promptformatierungen entsteht.*

Neben der Spezifikation der Rollen durch LangChain mit 'messages' gibt es eine weitere wichtige Regel zur Formatierung: das sogenannte Prompt-Format und die Verwendung von Tags.

Verschiedene LLMs können spezifische Prompt-Formate erfordern, einschließlich spezieller Tokens oder Tags, um die Eingaben korrekt zu strukturieren.

Eine korrekte Prompt-Formatierung stellt sicher, dass das LLM die Anweisungen und den Kontext richtig interpretiert. Falsch formatierte Prompts können zu unerwarteten oder falschen Ausgaben führen. Indem wir System-, Benutzer- und Assistenznachrichten klar definieren, ermöglichen wir es dem LLM, den Gesprächsfluss und seine Rolle darin besser zu verstehen.

Verschiedene Modelle (und ihre Varianten) verwenden unterschiedliche Prompt-Tags. 

Hier sind die Informationen für Llama 3.1: https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/

...zuerst als einfacher Prompt:

In [ ]:
result = llm_TogehterAI_Llama3_8B.invoke("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant that translates English to German. Please translate the user sentence.<|eot_id|><|start_header_id|>user<|end_header_id|>

I love Digital Humanities.<|eot_id|><|start_header_id|>assistant<|end_header_id|>""")
print(result.content)

...und noch mal mit zusätzlich mit LangChain spezialisierten Rollen:

In [ ]:
messages = [
    (
        "system", """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant that translates English to German. Please translate the user sentence.<|eot_id|><|start_header_id|>user<|end_header_id|>

"""
    ),
    ("human", """I love Digital Humanities.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""),
]
result = llm_TogehterAI_Llama3_8B.invoke(messages)
print(result.content)

# Prompt Templates

Jetzt wenden wir das Gleiche auf ein realistischeres Beispiel aus den Digital Humanities an.

Zusätzlich wird eine neue LangChain-Komponente eingeführt: die **Prompt Templates**. Prompt Templates helfen dabei, Prompts effizient zu verwalten und wiederzuverwenden. Durch die Verwendung von Platzhaltern innerhalb von Templates können wir Inhalte dynamisch einfügen, was unsere Prompts flexibel und anpassbar macht, ohne sie jedes Mal neu schreiben zu müssen.

LangChain bietet die Klassen `PromptTemplate` und `ChatPromptTemplate` an:

- `PromptTemplate` wird hauptsächlich verwendet, um einen einzelnen String zu formatieren und eignet sich für einfachere Eingaben.
- `ChatPromptTemplate` dient zur Formatierung einer Liste von Nachrichten.

Weitere Informationen gibt es hier: https://python.langchain.com/docs/concepts/#prompt-templates


#### String PromptTemplates


https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html

In [16]:
from langchain_core.prompts import PromptTemplate

Es gibt zwei Möglichkeiten, PromptTemplates zu initialisieren (siehe den API-Referenz-Link oben). Hier demonstrieren wir ausschließlich die von LangChain empfohlene Variante: die Initialisierung mit `from_template`.

Was machen wir?

1. Definition des Templates für den Prompt als Text.

2. Initialisierung einer PromptTemplate-Klasse:
    - Verwende das zuvor definierte `template_text`, um ein `PromptTemplate`-Objekt zu erstellen.

3. Festlegung des historischen Textes (Regeste), der bearbeitet werden soll.

4. Erstellung des finalen Prompts durch Einfügen der Regeste in das Template:
    - Übergabe der Regeste als Eingabevariable (`question`) an das Template.

5. Senden des erstellten Prompts an das Sprachmodell.

6. Rückgabe des Ergebnisses.


In [ ]:
template_text = """"<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>

Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

template = PromptTemplate.from_template(template_text)

regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ires in ehrn und tugent wolverhaltens halben Schloss und Herrschaft Ambras sambt allem pau, haus und vorrath, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und geben zu Prag den 3. tag des monats martii nach Christi unsers herrn geburt im 1564.

Eigenhändig unterschriebenes Or. Perg. mit abgefallenem Siegel, Urkunden des Familienarchivs."""

prompt = template.invoke({"question": regeste})

print(prompt)

In [ ]:
result = llm_TogehterAI_Llama3_8B.invoke(prompt)
print(result.content)

# ChatPromptTemplates

https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

Hier wird die gleiche Aufgabe mit einer `ChatPromptTemplate` realisiert.

In [22]:
from langchain_core.prompts import ChatPromptTemplate

Der Ablauf unterscheidet sich kaum von `PromptTemplate`:

1. Definition der Systemnachricht (`system_msg`).

2. Definition der Benutzernachricht (`user_msg`).

3. Erstellung eines `ChatPromptTemplate`-Objekt.

4. Festlegung des historischen Textes (Regeste), der bearbeitet werden soll.

5. Erzeugung des finalen Prompts.

6. Senden des erstellten Prompts an das Sprachmodell.

7. Rückgabe des Ergebnisses.


In [ ]:
system_msg = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>

"""
user_msg = """Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""


prompt_template = ChatPromptTemplate([
    ("system", system_msg),
    ("user", user_msg)
])


regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ires in ehrn und tugent wolverhaltens halben Schloss und Herrschaft Ambras sambt allem pau, haus und vorrath, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und geben zu Prag den 3. tag des monats martii nach Christi unsers herrn geburt im 1564.

Eigenhändig unterschriebenes Or. Perg. mit abgefallenem Siegel, Urkunden des Familienarchivs."""

prompt = prompt_template.invoke({"question": regeste})

print(prompt)

In [ ]:
print(f"{prompt.messages[0].content}{prompt.messages[1].content}")

In [ ]:
result = llm_TogehterAI_Llama3_8B.invoke(prompt)
print(result.content)

# Few-Shot Prompting

Eine häufig verwendete Technik, um die Leistung des Modells zu verbessern, ist das Einbinden von Beispielen in den Prompt. Diese Methode wird als Few-Shot-Prompting bezeichnet. Durch das Bereitstellen konkreter Beispiele erhält das Sprachmodell eine klare Vorstellung davon, wie es sich verhalten soll.

Um die Lesbarkeit des Codes zu verbessern, werden die Beispiele aus einer CSV-Datei eingelesen.

Wir werden im Prompt zwei Beispiele einfügen, die ebenfalls als Nachrichten mit definierten Rollen angegeben werden.

Pseudocode:

1. Öffnung der JSON-Datei mit den Beispielen und Laden des Inhalts in die Variable `examples`.

2. Definition der Systemnachricht (`system_msg`).

3. Definition der Benutzernachricht (`example_user_msg`) für die Beispiele.

4. Definition der Assistentennachricht (`example_assistant_msg`) für die Beispiele.

5. Definition der Benutzernachricht (`user_msg`) für die eigentliche Aufgabe.

6. Initialisierung eines `ChatPromptTemplate`-Objekts.

7. Speichern der Benutzereingaben und Assistentenantworten für Beispiel 1 und Beispiel 2 als Variablen.

8. Festlegung des historischen Textes (Regeste), der analysiert werden soll.

9. Erstellung des `variables`-Dictionaries:
    - Festlegen der Platzhalter-Werte für die beiden Beispiele und den Regestetext, die in das Template eingefügt werden.

10. Erzeugung des finalen Prompts.

11. Senden des erstellten Prompts an das Sprachmodell.

12. Rückgabe des Ergebnisses.


In [32]:
import json

with open('data/examples.json', 'r', encoding='utf-8') as file:
    examples = json.load(file)

system_msg = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

example_user_msg = """Please produce statements from the following text:
{example_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

example_assistant_msg = """{example_output}<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

user_msg = """Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

prompt_template = ChatPromptTemplate([
    ("system", system_msg),
    
    # First Example
    ("user", example_user_msg.replace("{example_input}", "{example_input_1}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_1}")),
    
    # Second Example
    ("user", example_user_msg.replace("{example_input}", "{example_input_2}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_2}")),
    
    # Actual User Message
    ("user", user_msg),
])

example_input_1 = examples[0]["user_input"]
example_output_1 = examples[0]["assistant_output_text"]
example_input_2 = examples[1]["user_input"]
example_output_2 = examples[1]["assistant_output_text"]

regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ihres in Ehren und Tugend wohlverhaltens halben Schloss und Herrschaft Ambras samt allem Bau, Haus und Vorrat, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und gegeben zu Prag den 3. Tag des Monats März nach Christi unseres Herrn Geburt im 1564.

Eigenhändig unterschriebenes Originalpergament mit abgefallenem Siegel, Urkunden des Familienarchivs."""

variables = {
    "example_input_1": example_input_1,
    "example_output_1": example_output_1,
    "example_input_2": example_input_2,
    "example_output_2": example_output_2,
    "question": regeste,
}

prompt = prompt_template.invoke(variables)


In [ ]:
for message in prompt.messages:
    print(message.content)

In [ ]:
result = llm_TogehterAI_Llama3_8B(prompt)

In [ ]:
print(result.content)

Die Antwort enthält jetzt weniger oder gar keinen einleitenden Text und keine Kommentare mehr. Nun wiederholen wir den Vorgang mit Beispielen im JSON-Format.

In [36]:
# Read Examples from JSON File
with open('data/examples.json', 'r', encoding='utf-8') as file:
    examples = json.load(file)

# System Message
system_msg = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

# User Message Template
example_user_msg = """Please produce statements from the following text:
{example_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# Assistant Message Template
example_assistant_msg = """{example_output}<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

# Actual User Message with Placeholder
user_msg = """Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

# The ChatPromptTemplate Including All Examples
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_msg),
    
    # First Example
    ("user", example_user_msg.replace("{example_input}", "{example_input_1}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_1}")),
    
    # Second Example
    ("user", example_user_msg.replace("{example_input}", "{example_input_2}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_2}")),
    
    # Actual User Message
    ("user", user_msg),
])

# First Example Input; from examles.json
example_input_1 = examples[0]["user_input"]

example_output_1 = examples[0]["assistant_output_json"]

# **Third Example Input**
example_input_2 = examples[1]["user_input"]

example_output_2 = examples[1]["assistant_output_json"]


# Actual Input Text
regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ihres in Ehren und Tugend wohlverhaltens halben Schloss und Herrschaft Ambras samt allem Bau, Haus und Vorrat, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und gegeben zu Prag den 3. Tag des Monats März nach Christi unseres Herrn Geburt im 1564.

Eigenhändig unterschriebenes Originalpergament mit abgefallenem Siegel, Urkunden des Familienarchivs."""

# Variables for Placeholders
variables = {
    "example_input_1": example_input_1,
    "example_output_1": example_output_1,
    "example_input_2": example_input_2,
    "example_output_2": example_output_2,
    "question": regeste,
}

# Invoke the Prompt with the Provided Inputs
prompt = prompt_template.invoke(variables)


In [ ]:
for message in prompt.messages:
    print(message.content)

In [ ]:
result = llm_TogehterAI_Llama3_8B(prompt)
print(result.content)

Der Output scheint im JSON-Format zu sein, doch wenn man den Objekttyp überprüft, stellt man fest, dass es sich um einen String handelt.

In [ ]:
print(type(result.content))

Es wäre jedoch wünschenswert, den Output direkt als JSON-Objekt zu erhalten. Ein JSON-Objekt ermöglicht eine einfachere Datenverarbeitung: Da es sich um strukturierte Daten handelt, lassen sich JSON-Objekte leicht im Code verarbeiten. Man kann direkt auf einzelne Felder zugreifen, ohne den Text vorher parsen zu müssen.Hierfür bietet LangChain die sogenannten **Output-Parsers** an. (Immer mehr Modelle unterstützen inzwischen das sogenannte "Function" oder "Tool Calling", das diesen Prozess automatisch übernimmt und es wird empfohlen, "Function/Tool Calling" anstelle von Output Parsing zu verwenden, wenn dies möglich ist.)

https://gorilla.cs.berkeley.edu/leaderboard.html


Die Formatierung des Outputs werden wir gemeinsam mit einer anderen LangChain-Komponente einführen, nämlich den **Chains**. Bevor wir jedoch damit weitermachen, noch ein kurzer Seitenkommentar zu den Beispielen. In unserem Fall haben wir nur zwei Beispiele, die fest im Prompt verankert sind. Für komplexere Anwendungsfälle kann es jedoch sinnvoll sein, mehrere Beispiele zu haben und dynamisch zwei oder drei davon auszuwählen, um sie dem Prompt hinzuzufügen. LangChain hat hierfür die sogenannten **Example Selectors** eingeführt. Diese Klassen sind dafür verantwortlich, Beispiele auszuwählen und in den Prompt zu integrieren (z.B. anhand der semantischen Ähnlichkeit zwischen unserem Input und den Beispielen). Weitere Informationen findest du hier: https://python.langchain.com/docs/how_to/#example-selectors

# Chains (+Output-Format)

Chains sind Abfolgen von Aufgaben, bei denen das Ergebnis einer Aufgabe als Eingabe für die nächste verwendet wird. Sie automatisieren Arbeitsabläufe, verbessern die Lesbarkeit des Codes und steigern die Effizienz.

Pseudocode:

1. Öffnung der JSON-Datei mit den Beispielen und Laden des Inhalts in die Variable `examples`.

2. Definition der Systemnachricht (`system_msg`).

3. Definition der Benutzernachricht (`example_user_msg`) für die Beispiele.

4. Definition der Assistentennachricht (`example_assistant_msg`) für die Beispiele.

5. Definition der Benutzernachricht (`user_msg`) für die eigentliche Aufgabe.

6. Initialisierung eines `ChatPromptTemplate`-Objekts.

7. Speichern der Benutzereingaben und Assistentenantworten für Beispiel 1 und Beispiel 2 als Variablen.

8. Festlegung des historischen Textes (Regeste), der analysiert werden soll.

9. Erstellung des `variables`-Dictionaries.

10. Verkettung von PromptTemplate und des Modells:
    - Eine Kette wird erstellt, die das `prompt_template` mit dem Modell `llm_TogetherAI_Llama3_8B` verbindet.

11. Ausführung der Kette mit den Variablen.

In [40]:
with open('data/examples.json', 'r', encoding='utf-8') as file:
    examples = json.load(file)

system_msg = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

example_user_msg = """Please produce statements from the following text:
{example_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

example_assistant_msg = """{example_output}<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

user_msg = """Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_msg),
    
    ("user", example_user_msg.replace("{example_input}", "{example_input_1}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_1}")),
    
    ("user", example_user_msg.replace("{example_input}", "{example_input_2}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_2}")),

    ("user", user_msg),
])


example_input_1 = examples[0]["user_input"]

example_output_1 = examples[0]["assistant_output_json"]

example_input_2 = examples[1]["user_input"]

example_output_2 = examples[1]["assistant_output_json"]

regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ihres in Ehren und Tugend wohlverhaltens halben Schloss und Herrschaft Ambras samt allem Bau, Haus und Vorrat, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und gegeben zu Prag den 3. Tag des Monats März nach Christi unseres Herrn Geburt im 1564.

Eigenhändig unterschriebenes Originalpergament mit abgefallenem Siegel, Urkunden des Familienarchivs."""

variables = {
    "example_input_1": example_input_1,
    "example_output_1": example_output_1,
    "example_input_2": example_input_2,
    "example_output_2": example_output_2,
    "question": regeste,
}

# prompt = prompt_template.invoke(variables) #ALT!

chain = prompt_template | llm_TogehterAI_Llama3_8B


In [ ]:
print(type(chain))

Eine Chain ist ein Objekt, das dazu dient, eine Abfolge von ausführbaren Aufgaben oder Aktionen darzustellen. Eine RunnableSequence definiert eine Abfolge von Operationen, die nacheinander ausgeführt werden sollen.

Mehr Informationen über Runnables in LangChain findest du hier: https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable

In [ ]:
print(chain)

In [ ]:
print(chain.first)
#print(chain.middle)
#print(chain.last)


In [ ]:
print(chain.first.input_variables) 
print(chain.first.messages) 

In [45]:
#result = llm_TogehterAI_Llama3_8B(prompt) #ALT!
result = chain.invoke(variables)

In [ ]:
print(result.content)

Es ist etwas schwieriger, sich den tatsächlichen Prompt anzeigen zu lassen, da dieser nie explizit erstellt wurde.

In [ ]:
for message in chain.first.messages:
    print(message.prompt.template)

In [ ]:
prompt_as_string = prompt_template.invoke(variables)

for message in prompt_as_string.messages:
    print(message.content)

...aber nun zurück zu unserem Output:


In [ ]:
print(result.content)

In [ ]:
print(type(result.content))

### Json Output Parser
Zunächst werden wir den einfachsten `JsonOutputParser` verwenden. Es ist wichtig zu betonen, dass der LLM-Output bereits in einer Form vorliegt, die die Aufgabe des Parsers erheblich erleichtert (dank der bereitgestellten Beispiele).

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser()

format_instructions = parser.get_format_instructions() #Provides instructions on how the LLM should format its output, based on the expected JSON schema.
print(format_instructions)

In [54]:
with open('data/examples.json', 'r', encoding='utf-8') as file:
    examples = json.load(file)


system_msg = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

example_user_msg = """Please produce statements from the following text:
{example_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

example_assistant_msg = """{example_output}<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

user_msg = """Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

prompt_template = ChatPromptTemplate([
    ("system", system_msg),
    
    ("user", example_user_msg.replace("{example_input}", "{example_input_1}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_1}")),
    
    ("user", example_user_msg.replace("{example_input}", "{example_input_2}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_2}")),
    
    ("user", user_msg),
])

# Bind format instructions
prompt_template = prompt_template.partial(format_instructions=format_instructions)

example_input_1 = examples[0]["user_input"]

example_output_1 = examples[0]["assistant_output_json"]

example_input_2 = examples[1]["user_input"]

example_output_2 = examples[1]["assistant_output_json"]


regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ihres in Ehren und Tugend wohlverhaltens halben Schloss und Herrschaft Ambras samt allem Bau, Haus und Vorrat, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und gegeben zu Prag den 3. Tag des Monats März nach Christi unseres Herrn Geburt im 1564.

Eigenhändig unterschriebenes Originalpergament mit abgefallenem Siegel, Urkunden des Familienarchivs."""

variables = {
    "example_input_1": example_input_1,
    "example_output_1": example_output_1,
    "example_input_2": example_input_2,
    "example_output_2": example_output_2,
    "question": regeste,
}

chain = prompt_template | llm_TogehterAI_Llama3_8B | parser


In [ ]:
print(chain)
print("--------------------")
print(chain.first.partial_variables)

In [56]:
result = chain.invoke(variables)

In [ ]:
print(type(result))

In [ ]:
print(result)

In [ ]:
print(json.dumps(result, indent=4, ensure_ascii=False))

## Custom Parser

Dafür werden wir den `PydanticOutputParser` verwenden. Dieser Parser basiert auf Pydantic, der am weitesten verbreiteten Bibliothek zur Datenvalidierung in Python. Mit dem `PydanticOutputParser` können wir den Output des Modells gemäß einem für unser Beispiel passenden Schema erzeugen.

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class Statement(BaseModel):
    text: str = Field(description="An informative sentence extracted from the historical text.")

class StatementGroup(BaseModel):
    date: str = Field(description="The date associated with the group of statements.")
    statements: List[Statement] = Field(description="A list of statements associated with the date.")

class AssistantOutput(BaseModel):
    statement_groups: List[StatementGroup] = Field(description="A list of statement groups, each associated with a date.")

parser = PydanticOutputParser(pydantic_object=AssistantOutput)

format_instructions = parser.get_format_instructions()
print(format_instructions)

In [61]:
with open('data/examples.json', 'r', encoding='utf-8') as file:
    examples = json.load(file)


system_msg = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a meticulous analyst with a focus on historical data. Your task is to process a provided historical record. For each given text please identify all statements that focus on the relationships and dependencies between entities in the given text, including biographical data. Also, structure the information, meaning:  Divide the text into two time based logical segments, each containing a self-contained piece of information (e.g., JJJJ-MM-TT, before JJJJ-MM-TT)
Return the statements in german as simple sentences that allow for easy interpretation and reconstruction of the historical context. Each sentence must be informative in itself. Please ensure to include names in each sentence as demonstrated:
John Doe goes to the store. John Doe buys apples.<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

example_user_msg = """Please produce statements from the following text:
{example_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

example_assistant_msg = """{example_output}<|eot_id|><|start_header_id|>user<|end_header_id|>
"""

user_msg = """Please produce statements from the following text:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

prompt_template = ChatPromptTemplate([
    ("system", system_msg),
    
    ("user", example_user_msg.replace("{example_input}", "{example_input_1}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_1}")),
    
    ("user", example_user_msg.replace("{example_input}", "{example_input_2}")),
    ("assistant", example_assistant_msg.replace("{example_output}", "{example_output_2}")),
    
    ("user", user_msg),
])

# Bind format instructions
prompt_template = prompt_template.partial(format_instructions=format_instructions)

example_input_1 = examples[0]["user_input"]

example_output_1 = examples[0]["assistant_output_json"]

example_input_2 = examples[1]["user_input"]

example_output_2 = examples[1]["assistant_output_json"]


regeste = """8652 1564 März 3, Prag.

Erzherzog Ferdinand schenkt der edlen Philippina Welserin sonderlich ihres in Ehren und Tugend wohlverhaltens halben Schloss und Herrschaft Ambras samt allem Bau, Haus und Vorrat, das ihm sein Vater Kaiser Ferdinand I. geschenksweise überlassen habe.

Geschehen und gegeben zu Prag den 3. Tag des Monats März nach Christi unseres Herrn Geburt im 1564.

Eigenhändig unterschriebenes Originalpergament mit abgefallenem Siegel, Urkunden des Familienarchivs."""

variables = {
    "example_input_1": example_input_1,
    "example_output_1": example_output_1,
    "example_input_2": example_input_2,
    "example_output_2": example_output_2,
    "question": regeste,
}

chain = prompt_template | llm_TogehterAI_Llama3_8B | parser


In [ ]:
print(chain.first.partial_variables)

In [ ]:
result_pydantic = chain.invoke(variables)
print(result_pydantic)

In [ ]:
print(type(result_pydantic))

In [ ]:
for statement_group in result_pydantic.statement_groups:
    print(statement_group.date)
    for statement in statement_group.statements:
        print(statement.text)

In [67]:
# Convert the AssistantOutput Pydantic object to a dictionary
result_dict = result_pydantic.dict()

# Convert the dictionary to a JSON string
json_output = json.dumps(result_dict, indent=4, ensure_ascii=False)

In [ ]:
print(json_output)

In [70]:
llm_TogehterAI_Llama3_70B_Lite = ChatTogether(
    model="meta-llama/Meta-Llama-3-70B-Instruct-Lite",
    temperature=0,
)

In [ ]:
chain = prompt_template | llm_TogehterAI_Llama3_70B_Lite | parser
result_pydantic = chain.invoke(variables)
result_dict = result_pydantic.dict()
json_output = json.dumps(result_dict, indent=4, ensure_ascii=False)
print(json_output)